In [1]:
import os
import boto3

In [2]:
# Configura tu cliente de S3
s3 = boto3.client('s3')
bucket_name = 'datainternetaccess-cleaning'
s3_folder = '' 

In [3]:
# Define la ruta de tu carpeta local
local_folder = 'Limpieza/data'

In [4]:
# Función para obtener archivos de S3
def get_s3_files(bucket_name, s3_folder):
    s3_files = []
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)
    if 'Contents' in response:
        for obj in response['Contents']:
            # Obtenemos los nombres de los archivos en S3
            s3_files.append(obj['Key'].replace(s3_folder + '/', ''))
    return s3_files


In [5]:
# Función para obtener archivos locales
def get_local_files(local_folder):
    local_files = []
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            # Obtenemos la ruta relativa para comparar con los nombres de S3
            relative_path = os.path.relpath(os.path.join(root, file), local_folder)
            local_files.append(relative_path)
    return local_files

In [6]:
# Compara los archivos locales con los archivos de S3
def upload_new_files(bucket_name, local_folder, s3_folder):
    s3_files = get_s3_files(bucket_name, s3_folder)
    local_files = get_local_files(local_folder)

    # Archivos que están en la carpeta local pero no en S3
    new_files = set(local_files) - set(s3_files)

    for file in new_files:
        # Subir el archivo a S3
        local_path = os.path.join(local_folder, file)
        s3_key = f"{s3_folder}/{file}"
        print(f"Subiendo {file} a {s3_key}")
        s3.upload_file(local_path, bucket_name, s3_key)

In [7]:
# Llamamos la función para subir los archivos nuevos
upload_new_files(bucket_name, local_folder, s3_folder)

Subiendo DatasetCoordenadas.csv a /DatasetCoordenadas.csv
Subiendo DatasetCoordenadas.csv:Zone.Identifier a /DatasetCoordenadas.csv:Zone.Identifier


In [8]:
# Compara los archivos de S3 con los archivos locales
def download_new_files(bucket_name, local_folder, s3_folder):
    s3_files = get_s3_files(bucket_name, s3_folder)
    local_files = get_local_files(local_folder)

    # Archivos que están en S3 pero no en la carpeta local
    new_files = set(s3_files) - set(local_files)

    for file in new_files:
        # Descargar el archivo desde S3
        local_path = os.path.join(local_folder, file)
        s3_key = f"{s3_folder}/{file}"

        # Crear los directorios locales si no existen
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        print(f"Descargando {file} desde {s3_key} a {local_path}")
        s3.download_file(bucket_name, s3_key, local_path)

In [9]:
# Llamamos la función para descargar los archivos nuevos
download_new_files(bucket_name, local_folder, s3_folder)